In [141]:
#initial imports
import requests
import pandas as pd
import xlwings as xw
import time as tm
import yfinance as yf
from datetime import datetime


url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"

headers = {"Accept-Encoding":"gzip, deflate, br",
           "Accept-Language":"en-US,en;q=0.9",
           "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (HTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

session = requests.Session()
data  = session.get(url,headers=headers).json()["records"]["data"]

ocdata = []

for i in data:
    for j,k in i.items():
        if j == "CE" or j == 'PE':
            info = k
            info["instrument_type"] = j
            ocdata.append(info)


In [142]:
# live nifty price
nifty_ticker = yf.Ticker("^NSEI")
current_price = nifty_ticker.history(period = "id")["Close"].iloc[0]
current_price


19674.25

In [143]:
#creating Data dataframe and sorting accourding to date 
oc = pd.DataFrame(ocdata)
oc_sorted = oc.sort_values(by='expiryDate')


In [144]:
#Making copy of original sorted df and sort ce and pe
df = oc_sorted.copy()
group = df.groupby('instrument_type')
ce_group = group.get_group('CE')
pe_group = group.get_group('PE')
#required columns
CE_col = ['openInterest','changeinOpenInterest','totalTradedVolume','lastPrice','strikePrice','expiryDate']
PE_col = ['strikePrice','lastPrice','totalTradedVolume','changeinOpenInterest','openInterest','expiryDate']
ce_group = ce_group[CE_col]
pe_group = pe_group[PE_col]
# Merge CE and PE options for the specified expiryDate
df1 = pd.merge(ce_group, pe_group, on=['strikePrice', 'expiryDate'], suffixes=("_CE", '_PE'), how='inner')



In [148]:
#making proper date format
df1['expiryDate'] = pd.to_datetime(df1['expiryDate'],format='%d-%b-%Y')
nearest_expiry = df1['expiryDate'].min()
#nearest expiry
formatted_date = nearest_expiry.strftime('%d-%b-%Y')
present_week_expiry= formatted_date
present_week_expiry


'28-Sep-2023'

In [146]:
#finding the nearest expiry
df1 = df1[df1['expiryDate'] == present_week_expiry]
df1 = df1.sort_values(by='strikePrice',ascending=True)
df1

,openInterest_CE,changeinOpenInterest_CE,totalTradedVolume_CE,lastPrice_CE,strikePrice,expiryDate,lastPrice_PE,totalTradedVolume_PE,changeinOpenInterest_PE,openInterest_PE
403,4.0,2,4,7729.80,12000,2023-09-28,0.40,70236,7396,8759.0
402,8256.0,-599,614,6710.60,13000,2023-09-28,0.45,24193,54,5812.0
401,190.0,-3,19,5722.25,14000,2023-09-28,0.40,15632,1710,3131.0
399,1679.0,-74,116,4701.40,15000,2023-09-28,0.40,7552,-142,2252.0
404,5168.0,-320,521,3710.00,16000,2023-09-28,0.65,7595,468,12136.0
...,...,...,...,...,...,...,...,...,...,...
405,5441.0,-330,29568,1.25,21650,2023-09-28,0.00,0,0,0.0
393,11886.0,1140,45253,1.20,21700,2023-09-28,0.00,0,0,0.0
396,24210.0,1909,65216,1.25,22000,2023-09-28,2286.00,756,-221,8156.0
398,26583.0,846,86291,1.05,23000,2023-09-28,3289.00,455,-319,2708.0


In [147]:
#To find ATM(at the money)
target_value = current_price

df1['difference'] = abs(df1['strikePrice']-target_value )
diff = df1.loc[df1['difference'].idxmin()]
d = diff['strikePrice']

atm = df1[df1['strikePrice'] == d]
atm


,openInterest_CE,changeinOpenInterest_CE,totalTradedVolume_CE,lastPrice_CE,strikePrice,expiryDate,lastPrice_PE,totalTradedVolume_PE,changeinOpenInterest_PE,openInterest_PE,difference
429,14591.0,9969,488289,127.6,19650,2023-09-28,71.0,1991804,33484,57932.0,24.25
